In [5]:
# ==========================================
# 0. 라이브러리 import
# ==========================================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report

from xgboost import XGBClassifier
import optuna


In [6]:
df = pd.read_csv("stroke.csv")

In [7]:
# ==========================================
# 1. 데이터 준비 (df가 이미 있다고 가정)
# ==========================================
target_col = "stroke"  # <- 타깃 컬럼명 확인

X = df.drop(columns=[target_col])
y = df[target_col]


In [8]:
# ==========================================
# 2. Train/Test Split (stratify로 클래스 비율 유지)
# ==========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [9]:
# ==========================================
# 3. SMOTE 대신 scale_pos_weight 계산 (불균형 가중치)
#    - pos_ratio: 양성(1) 비율
#    - scale_pos_weight = (음성/양성)
# ==========================================
pos_ratio = y_train.sum() / len(y_train)
scale_pos = (1 - pos_ratio) / pos_ratio

print(f"양성(1) 비율: {pos_ratio:.4f}")
print(f"scale_pos_weight: {scale_pos:.4f}")


양성(1) 비율: 0.0487
scale_pos_weight: 19.5427


In [10]:
# ==========================================
# 4. Optuna 목적함수 정의 (가중치 기반 XGBoost + AUC CV)
#    - SMOTE 없음 (원본 분포 유지)
#    - StratifiedKFold로 CV에서도 클래스 비율 유지
# ==========================================
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

def objective(trial):

    params = {
        # 모델이 학습하는 속도(작을수록 천천히 안정적으로)
        "learning_rate": trial.suggest_float("lr", 0.01, 0.1),

        # 트리의 복잡도(깊이)
        "max_depth": trial.suggest_int("max_depth", 3, 6),

        # 트리 개수
        "n_estimators": trial.suggest_int("n_estimators", 300, 900),

        # 일부 데이터만 사용해 과적합 줄이기
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),

        # 일부 변수만 사용해 특정 변수 쏠림 방지
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),

        # 너무 작은 그룹으로 split 하지 않도록(과적합 방지)
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),

        # 의미 있는 경우에만 split(잡음 감소)
        "gamma": trial.suggest_float("gamma", 0, 1),

        # ✅ SMOTE 대신 가중치로 불균형 처리
        "scale_pos_weight": scale_pos,

        # 평가 지표
        "eval_metric": "auc",

        # 재현성/속도
        "random_state": 42,
        "n_jobs": -1,

        # (GPU 쓰는 환경이면 켜기)
        "tree_method": "gpu_hist",
        "predictor": "gpu_predictor",
    }

    model = XGBClassifier(**params)

    auc = cross_val_score(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring="roc_auc"
    ).mean()

    return auc


In [11]:
# 최적 파라미터(Optuna 결과)
best_params = {
    "learning_rate": 0.010002154204346032,
    "max_depth": 4,
    "n_estimators": 537,
    "subsample": 0.6055566573624598,
    "colsample_bytree": 0.8979287201002785,
    "min_child_weight": 5,
    "gamma": 0.7209404101015859,
}

In [12]:
# ==========================================
# 6. Best params로 최종 모델 학습 (Train 전체로 fit)
#    - Optuna에서 "lr" 이름으로 뽑았으니, XGBoost 파라미터명으로 변환 필요
# ==========================================

# Optuna 탐색에 없었던 고정값들 추가
best_params.update({
    "scale_pos_weight": scale_pos,
    "eval_metric": "auc",
    "random_state": 42,
    "n_jobs": -1,
    "tree_method": "gpu_hist",
    "predictor": "gpu_predictor",
})

final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)


/home/kdt_43/miniconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:37:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/kdt_43/miniconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:37:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8979287201002785, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0.7209404101015859,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.010002154204346032,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=537, n_jobs=-1,
              num_parallel_tree=None, predictor='gpu_predictor', ...)

In [13]:
# ==========================================
# 7. Test 평가 (ROC-AUC + 기본 threshold 성능)
# ==========================================
y_proba = final_model.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_proba)

print(f"✅ Test ROC-AUC: {test_auc:.4f}")

# threshold=0.5는 참고용
y_pred = (y_proba >= 0.4).astype(int)
print(classification_report(y_test, y_pred, digits=3))


✅ Test ROC-AUC: 0.8415
              precision    recall  f1-score   support

           0      0.987     0.727     0.838       972
           1      0.134     0.820     0.230        50

    accuracy                          0.732      1022
   macro avg      0.561     0.774     0.534      1022
weighted avg      0.946     0.732     0.808      1022



/home/kdt_43/miniconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:37:59] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/kdt_43/miniconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:37:59] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
